# 🔧 Model Tuning with GridSearchCV and RandomizedSearchCV

When we train a machine learning model, we often need to adjust **hyperparameters** (settings chosen before training) to get the best performance.  
Examples:
- Decision Tree: `max_depth`, `criterion`
- Random Forest: `n_estimators`, `max_features`
- SVM: `C`, `kernel`

👉 Finding the best hyperparameters is called **Hyperparameter Tuning**.  
Two common methods in Scikit-learn:
1. **GridSearchCV** → tries all possible combinations (exhaustive search).  
2. **RandomizedSearchCV** → tries a fixed number of random combinations (faster).  

Both methods use **cross-validation** internally to ensure results are reliable.  


## 🔍 GridSearchCV

- **Grid Search** means searching exhaustively through a manually specified set of hyperparameter values.  
- For each combination of hyperparameters:
  1. Model is trained
  2. Evaluated using cross-validation
  3. Best parameters are chosen based on performance metric

✅ Advantage: Finds the best combination.  
❌ Disadvantage: Can be very slow if parameter grid is large.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
df=sns.load_dataset('iris')

In [19]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x = df.drop('species', axis=1)
y = df['species']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier()
model_knn.fit(x_train, y_train)

KNeighborsClassifier()

In [26]:
model_knn.score(x_test,y_test)

1.0

In [27]:
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(x_train, y_train)

SVC()

In [28]:
model_svm.score(x_test,y_test)

1.0

In [29]:
from sklearn.model_selection import GridSearchCV

In [31]:
classifier = GridSearchCV((model_svm),{
    'C' : [1,10,20,30],
    'kernel' :['rbf','linear'],
},cv = 5,return_train_score = False)

In [33]:
classifier.fit(x,y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 10, 20, 30], 'kernel': ['rbf', 'linear']})

In [34]:
classifier.cv_results_

{'mean_fit_time': array([0.01028185, 0.00662255, 0.01398759, 0.00497761, 0.01047568,
        0.00479832, 0.0040113 , 0.00486684]),
 'std_fit_time': array([0.00492771, 0.00308402, 0.00534929, 0.00227764, 0.00285243,
        0.00195153, 0.00024089, 0.00068349]),
 'mean_score_time': array([0.00996065, 0.00676975, 0.00622783, 0.00512967, 0.00486441,
        0.00288944, 0.00306201, 0.00764914]),
 'std_score_time': array([0.00618382, 0.00539162, 0.00234748, 0.00305783, 0.00286603,
        0.00011541, 0.00020991, 0.00606591]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C

In [35]:
results = pd.DataFrame(classifier.cv_results_)

In [36]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010282,0.004928,0.009961,0.006184,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,6
1,0.006623,0.003084,0.006770,0.005392,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.013988,0.005349,0.006228,0.002347,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.004978,0.002278,0.005130,0.003058,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.010476,0.002852,0.004864,0.002866,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
5,0.004798,0.001952,0.002889,0.000115,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
6,0.004011,0.000241,0.003062,0.000210,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.000000,0.933333,0.966667,1.0,0.973333,0.024944,4
7,0.004867,0.000683,0.007649,0.006066,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.900000,1.0,0.960000,0.048990,8


In [37]:
results[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.966667
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.980000
5,20,linear,0.966667
6,30,rbf,0.973333
7,30,linear,0.960000


## 🎲 RandomizedSearchCV

- Instead of trying **all** combinations, Randomized Search picks a **random subset** of hyperparameter combinations.  
- You can control how many iterations it should try using `n_iter`.

✅ Advantage: Much faster than Grid Search for large parameter spaces.  
❌ Disadvantage: Might miss the absolute best combination (but usually finds near-optimal).

In [38]:
from sklearn.model_selection import RandomizedSearchCV

In [39]:
classifier_r = RandomizedSearchCV((model_svm),{
    'C' : [1,10,20,30],
    'kernel' :['rbf','linear'],
},n_iter = 4,cv = 5,return_train_score = False)

In [41]:
classifier_r.fit(x,y)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=4,
                   param_distributions={'C': [1, 10, 20, 30],
                                        'kernel': ['rbf', 'linear']})

In [42]:
results = pd.DataFrame(classifier_r.cv_results_)

In [43]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004180,0.000303,0.003279,0.000477,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.007761,0.006159,0.003134,0.000161,rbf,20,"{'kernel': 'rbf', 'C': 20}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.004038,0.000250,0.002954,0.000079,linear,10,"{'kernel': 'linear', 'C': 10}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
3,0.005935,0.003491,0.008512,0.007750,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1


In [44]:
results[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,20,rbf,0.980000
2,10,linear,0.973333
3,10,rbf,0.980000


# ✅ Conclusion

- **GridSearchCV** → checks all combinations, best for small parameter spaces.  
- **RandomizedSearchCV** → samples random combinations, faster for large parameter spaces.  
- Both use **cross-validation** internally for reliable performance estimation.  

👉 In practice:
- Start with **RandomizedSearchCV** to explore quickly.  
- Then refine using **GridSearchCV** on a smaller narrowed-down grid.